# Human Traffic of Stadium

### Description

Table: Stadium

| Column Name   | Type    |
|---------------|---------|
| id            | int     |
| visit_date    | date    |
| people        | int     |

visit_date is the column with unique values for this table.
Each row of this table contains the visit date and visit id to the stadium with the number of people during the visit.
As the id increases, the date increases as well.
 
Write a solution to display the records with three or more rows with consecutive id's, and the number of people is greater than or equal to 100 for each.

Return the result table ordered by visit_date in ascending order.

### Imports

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, DateType
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from datetime import datetime

### Sample data

In [ ]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("visit_date", DateType(), True),
    StructField("people", IntegerType(), True)
])

data = [
    (1, datetime.strptime("2017-01-01", "%Y-%m-%d"), 10),
    (2, datetime.strptime("2017-01-02", "%Y-%m-%d"), 109),
    (3, datetime.strptime("2017-01-03", "%Y-%m-%d"), 150),
    (4, datetime.strptime("2017-01-04", "%Y-%m-%d"), 99),
    (5, datetime.strptime("2017-01-05", "%Y-%m-%d"), 145),
    (6, datetime.strptime("2017-01-06", "%Y-%m-%d"), 1455),
    (7, datetime.strptime("2017-01-07", "%Y-%m-%d"), 199),
    (8, datetime.strptime("2017-01-09", "%Y-%m-%d"), 188)
]

spark.createDataFrame(data, schema).createOrReplaceTempView("Stadium")

display(spark.table("Stadium"))


### Solution

In [ ]:
window = Window.orderBy("visit_date")

stadium_enriched = (
    spark.read.table("Stadium")
        .withColumn("one_prev", F.lag("people", 1).over(window))
        .withColumn("two_prev", F.lag("people", 2).over(window))
        .withColumn("one_next", F.lead("people", 1).over(window))
        .withColumn("two_next", F.lead("people", 2).over(window))
)

result = (
    stadium_enriched.filter(
        (F.col("people") >= 100) & (
            ((F.col("one_prev") >= 100) & (F.col("two_prev") >= 100)) |
            ((F.col("one_next") >= 100) & (F.col("two_next") >= 100)) |
            ((F.col("one_prev") >= 100) & (F.col("one_next") >= 100))
        ))
        .select("id", "visit_date", "people")
        .orderBy("visit_date")
)

display(result)